In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob, pickle, time, gc, copy, sys
import warnings
import cv2
import multiprocessing

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
sys.path.append('../src')
from utils import ri, pickle_load, pickle_save
from preprocess_fold import data_split_StratifiedKFold

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import _LRScheduler

from sklearn import metrics
import timm
from utils_pytorch import cycle, CosineLR, AverageMeter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = True

In [3]:
from albumentations import (
    Compose,
    RandomCrop,
    RandomBrightnessContrast,
    ShiftScaleRotate,
    Cutout,
)
from albumentations.core.transforms_interface import ImageOnlyTransform

# Config

In [4]:
# config
DEBUG = True # set False to do all process
preprocess_dir = "../input/preprocess"
output_path = "../output"
df_train_path = "{}/df_train.csv".format(preprocess_dir)
train_npy_dir_path = "{}/train_law_npy".format(preprocess_dir)
os.makedirs(output_dir, exist_ok=True)

In [5]:
NUM_FOLD = 5
NUM_EPOCH = 16
NUM_CYCLE = 16
BATCH_SIZE = 80
FOLD_LIST = [1,2,3,4,5] 
LR_RANGE = [1e-3, 1e-4]
STEP_PER_EPOCH = 512
REDUCE_RATE = 0.1
MODEL_NAME = 'b0' # b0 or b2
if DEBUG:
    NUM_EPOCH = 1
    NUM_CYCLE = 1
    FOLD_LIST = [1]

In [6]:
col_index = 'SOPInstanceUID'
col_groupby = 'StudyInstanceUID'
col_targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
    'pe_present_on_image',
]
NUM_CLASS = len(col_targets)
print('NUM_CLASS: {}'.format(NUM_CLASS))

NUM_CLASS: 10


# Data Loading

In [7]:
# load train data
df_train = pd.read_csv(df_train_path)
print(df_train.shape)
df_train.head()

(1790594, 27)


,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
1,6897fa9de148,2bfbb7fd2e8b,52b6b0b793bb,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,1.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
2,6897fa9de148,2bfbb7fd2e8b,1997c99c9d59,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,2.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
3,6897fa9de148,2bfbb7fd2e8b,c6f29ac6659b,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,3.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
4,6897fa9de148,2bfbb7fd2e8b,487d9ab5531f,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,4.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0


# Preprocessing

In [8]:
# make exam-level train data
df_train_exam = ri(df_train[df_train[col_groupby].duplicated()==False])
print(df_train_exam.shape)
df_train_exam.head()

(7279, 27)


,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.338710,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
1,013358b540bb,2805267980e7,c0e16bbe96ef,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,145,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,1
2,0cee26703028,bac7becd2970,64bf37f1a302,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,144,17.0,0.118056,/mnt/disks/data6/rsna2020/preprocessed/train_l...,2
3,c28f3d01b14f,7d17c72fd0ce,265035a0f7aa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,105,6.0,0.057143,/mnt/disks/data6/rsna2020/preprocessed/train_l...,3
4,c8fbf1e08ac5,275497911f02,7a33bbba8386,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,81,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,4


In [9]:
# set image file paths
df_train['npy_path'] = (
    train_npy_dir_path + "/"
    + df_train[col_index] + ".npy"
)
print(df_train['npy_path'][0])
df_train.head()

/mnt/disks/data4/rsna2020/preprocessed/train_law_npy/baedb900c69c.npy


,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.33871,/mnt/disks/data4/rsna2020/preprocessed/train_l...,0
1,6897fa9de148,2bfbb7fd2e8b,52b6b0b793bb,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,1.0,124,42.0,0.33871,/mnt/disks/data4/rsna2020/preprocessed/train_l...,0
2,6897fa9de148,2bfbb7fd2e8b,1997c99c9d59,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,2.0,124,42.0,0.33871,/mnt/disks/data4/rsna2020/preprocessed/train_l...,0
3,6897fa9de148,2bfbb7fd2e8b,c6f29ac6659b,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,3.0,124,42.0,0.33871,/mnt/disks/data4/rsna2020/preprocessed/train_l...,0
4,6897fa9de148,2bfbb7fd2e8b,487d9ab5531f,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,4.0,124,42.0,0.33871,/mnt/disks/data4/rsna2020/preprocessed/train_l...,0


# Data Splitting

In [10]:
# data splitting stratified by 3 classes, PE positive, PE negative or indeterminate
df_tmp = df_train_exam
df_tmp['3class'] = 1 # PE positive
df_tmp['3class'][df_tmp['negative_exam_for_pe']==1] = 0 # PE negative
df_tmp['3class'][df_tmp['indeterminate']==1] = 2 # indeterminate
df_fold_exam = data_split_StratifiedKFold(df_tmp, col_groupby, col_stratified='3class') # apply stratified K-fold
df_fold = pd.merge(df_train[[col_groupby]], df_fold_exam, on=col_groupby, how='left')
print(df_fold.shape)
print(df_fold_exam.shape)
df_fold_exam.head()

(1790594, 11)
(7279, 11)


,StudyInstanceUID,fold1_train,fold1_valid,fold2_train,fold2_valid,fold3_train,fold3_valid,fold4_train,fold4_valid,fold5_train,fold5_valid
0,6897fa9de148,1,0,0,1,1,0,1,0,1,0
1,013358b540bb,0,1,1,0,1,0,1,0,1,0
2,0cee26703028,1,0,1,0,0,1,1,0,1,0
3,c28f3d01b14f,1,0,1,0,1,0,1,0,0,1
4,c8fbf1e08ac5,0,1,1,0,1,0,1,0,1,0


# Definition of Model

In [11]:
class nnWindow(nn.Module): # window setting optimization layer
    def __init__(self):
        super(nnWindow, self).__init__()
        wso = np.array(((40,80),(80,200),(40,400)))/1000
        conv_ = nn.Conv2d(1,3, kernel_size=(1, 1))
        conv_.weight.data.copy_(torch.tensor([[[[1./wso[0][1]]]],[[[1./wso[1][1]]]],[[[1./wso[2][1]]]]]))
        conv_.bias.data.copy_(torch.tensor([0.5 - wso[0][0]/wso[0][1],
                                            0.5 - wso[1][0]/wso[1][1],
                                            0.5 -wso[2][0]/wso[2][1]]))
        self.window = nn.Sequential(
            conv_,
            nn.Sigmoid(),
            nn.InstanceNorm2d(3)
        )
    def forward(self, input1):
        return self.window(input1)
        
        
class CNN_2D(nn.Module):
    def __init__(self, num_classes=2, base_model='tf_efficientnet_b0_ns'):
        super(CNN_2D, self).__init__()

        self.num_classes = num_classes
        self.mode = 'train'
        self.window = nnWindow()
        self.base_model = timm.create_model(base_model, pretrained=True, num_classes=10).cuda()
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.last_linear = nn.Linear(self.base_model.num_features, num_classes)

    def forward(self, input1):
        bs, ch, h, w = input1.size()
        x = self.window(input1)
        x = self.base_model.forward_features(x)
        feature = self.avgpool(x).view(bs, -1)
        y = self.last_linear(feature)

        return y

    def feature(self, input1):
        bs, ch, h, w = input1.size()
        x = self.window(input1)
        x = self.base_model.forward_features(x)
        feature = self.avgpool(x).view(bs, -1)
        y = self.last_linear(feature)

        return y, feature

# Definition of Training

In [12]:
# define augmentation
def get_brightness_contrast(x, brightness_std=0.1, contrast_std=0.1):
    alpha = np.clip(1 + np.random.normal()*brightness_std, 0.5, 2)
    beta = np.clip(1 + np.random.normal()*contrast_std, 0.5, 2)
    mean = x.mean()
    x_new = (x- mean[np.newaxis, np.newaxis]) * beta + mean[np.newaxis, np.newaxis]*alpha
    return x_new

class BrightNessContrastMy(ImageOnlyTransform):
    def apply(self, img, **params):
        return get_brightness_contrast(img)

In [13]:
# define dataset
class ImageDataset(Dataset):
    def __init__(self, X_image, X_exam, y, transform=None, mixup=(0, 0), verbose=False):
        self.X_image = X_image
        self.X_exam = X_exam
        self.y = y
        self.transform = transform
        self.mixup = mixup
        self.verbose = verbose

    def do_mixup(self, x_image, q, target):
        alpha = self.mixup[1]
        idx2 = np.random.randint(len(self.X_image))
        img_path = self.X_image['npy_path'][idx2]
        index_study = self.X_image['exam_index'][idx2]
        x_image2 = np.load(img_path).astype(np.float32).reshape([512,512,1])
        q2 = self.X_image['q_i'][idx2].astype(np.float16).reshape([1])
        M = self.X_exam['RescaleSlope'][index_study]
        B = self.X_exam['RescaleIntercept'][index_study]
        x_image2 = x_image2 * M + B
        target2 = self.y[idx2].astype(np.float16) #.reshape([1])
        if self.transform is not None:
            x_image2 = self.transform(image=x_image2)['image']

        rate = np.random.beta(alpha,alpha)
        x_image = x_image*rate + x_image2*(1-rate)
        q = q*rate + q2*(1-rate)
        target = target*rate + target2*(1-rate)
        return x_image, q, target
    
    def __getitem__(self, index):
        img_path = self.X_image['npy_path'][index]
        index_study = self.X_image['exam_index'][index]
        x_image = np.load(img_path).astype(np.float32).reshape([512,512,1])
        if self.verbose: print(x_image.shape)
        q = self.X_image['q_i'][index].astype(np.float16).reshape([1])
        M = self.X_exam['RescaleSlope'][index_study]
        B = self.X_exam['RescaleIntercept'][index_study]
        x_image = x_image * M + B
        if self.verbose: print(x_image.mean(), x_image.std())
        if self.verbose: print(x_image.shape)
        
        target = self.y[index].astype(np.float16) #.reshape([1])
        if self.transform is not None:
            x_image = self.transform(image=x_image)['image']
        x_image = (x_image.transpose([2,0,1])/1000).astype(np.float16)
        return x_image, target, q
    
    def __len__(self):
        return len(self.X_image)

In [14]:
# set class weights
# exam-level weigths is equal to the competition metric
# (sum of exam-level):image-level = 1:1
class_weights = np.array([
    0.0736196319, 
    0.09202453988, 
    0.1042944785, 
    0.1042944785, 
    0.1877300613, 
    0.06257668712, 
    0.06257668712,
    0.2346625767,
    0.0782208589,
    1
], np.float32) / 2

class_weights = torch.from_numpy(class_weights).cuda()

In [15]:
# define training
def train(loader, model, optimizer, scheduler, num_step=-1, verbose=10):
    if num_step==-1: num_step = len(loader)
    loss1_avr = AverageMeter()
    criterion1 = nn.BCEWithLogitsLoss(weight=class_weights, reduce=False).cuda()
    lastfunc = nn.Sigmoid().cuda()
    scaler = torch.cuda.amp.GradScaler() 
    
    # switch to train mode
    model .train()

    starttime = time.time()
    # training
    for i, (input1, target, _) in enumerate(loader):
        optimizer.zero_grad()
        if i+1>num_step: break
        input1 = torch.autograd.Variable(input1.to(device, non_blocking=True)) #.half()
        target = torch.autograd.Variable(target.to(device, non_blocking=True)) #.half()
        with torch.cuda.amp.autocast(): # mix-precision
            output = model(input1)
            loss = criterion1(output, target) # calc weighted BCE loss
            loss = torch.sum(loss, axis=1)
            loss = torch.mean(loss)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # record loss
        loss1_avr.update(loss.data, input1.size(0))
        
        # display log
        if (i+1)%verbose==0 or i+1==num_step:
            print("Step: {}/{} ".format(i + 1, num_step)
                  + "BCE 1: {:.3f} ".format(loss1_avr.avg.item())
                  + "Sec: {:.1f} ".format(time.time()-starttime)
                  )
            
    return loss1_avr.avg.item()

In [16]:
# define validation
def validate(loader, model, verbose=10):
    loss1_avr = AverageMeter()
    criterion1 = nn.BCEWithLogitsLoss(weight=class_weights, reduce=False).cuda()
    lastfunc = nn.Sigmoid().cuda()

    # switch to eval mode
    model .eval()
    
    starttime = time.time()
    # training
    preds = np.zeros([0, NUM_CLASS], np.float32)
    for i, (input1, target, _) in enumerate(loader):
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                input1 = input1.to(device, non_blocking=True)
                target = target.to(device, non_blocking=True)
                output = model(input1)
                loss = criterion1(output, target)
                loss = torch.sum(loss, axis=1)
                loss = torch.mean(loss)
                pred = lastfunc(output)

        # record loss
        loss1_avr.update(loss.data, input1.size(0))
        preds = np.concatenate([preds, pred.data.cpu().numpy()])

        # display log
        if (i+1)%verbose==0 or i+1==len(loader):
            print("Step: {}/{} ".format(i + 1, len(loader))
                  + "BCE 1: {:.3f} ".format(loss1_avr.avg.item())
                  + "Sec: {:.1f} ".format(time.time()-starttime)
                  )
        
    return loss1_avr.avg.item(), preds

In [17]:
transformations = Compose([
    ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, interpolation=1, border_mode=0),
    RandomCrop(int(512*7/8), int(512*7/8)),
    BrightNessContrastMy(p=1),
    Cutout(max_h_size=512//8, max_w_size=512//8),
])

# Training

In [18]:
# set train log
log_columns = ['epoch', 'loss1', 'val_loss1', 'time']
for col in col_targets:
    log_columns.append("val_bce_{}".format(col))
    log_columns.append("val_auc_{}".format(col))

In [19]:
# training
for fold in range(NUM_FOLD):
    starttime = time.time()
    if fold+1 not in FOLD_LIST: continue
    print("fold: {}".format(fold + 1))
    
    # reduce validation data to save time
    np.random.seed(42)
    val_reduce = np.random.rand(np.sum(df_fold['fold{}_valid'.format(fold+1)]==1))<REDUCE_RATE

    # build model
    if MODEL_NAME=='b0':
        base_model = 'tf_efficientnet_b0_ns'
    elif MODEL_NAME=='b2':
        base_model = 'tf_efficientnet_b2_ns'
    model  = CNN_2D(base_model=base_model, num_classes=NUM_CLASS).cuda()
    
    # train dataset
    X_train_image = ri(df_train[df_fold['fold{}_train'.format(fold+1)]==1])
    y_train = df_train[col_targets][df_fold['fold{}_train'.format(fold+1)]==1].values
    dataset_train = ImageDataset(X_train_image, df_train_exam, y_train, transformations, mixup=(0.5, 0.3))
    train_loader = DataLoader(dataset_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=multiprocessing.cpu_count(),
                              pin_memory=True,
                              worker_init_fn=lambda x: np.random.seed(),
                              )

    # valid dataset
    X_valid_image = ri(ri(df_train[df_fold['fold{}_valid'.format(fold+1)]==1].iloc[val_reduce]))
    y_valid = df_train[col_targets][df_fold['fold{}_valid'.format(fold+1)]==1].values[val_reduce]
    dataset_valid = ImageDataset(X_valid_image, df_train_exam, y_valid)
    valid_loader = DataLoader(dataset_valid,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=multiprocessing.cpu_count(),
                              pin_memory=True,
                              )
    
    # set optimizer and loss
    optimizer = optim.Adam(model.parameters(), lr=LR_RANGE[0])
    scheduler = CosineLR(optimizer, step_size_min=LR_RANGE[1], t0=STEP_PER_EPOCH * NUM_CYCLE, tmult=1)
    
    # set train log dataframe
    train_log = pd.DataFrame(columns=log_columns)
    
    # training
    for epoch in range(NUM_EPOCH):
        loss1 = train(train_loader, model, optimizer, scheduler, num_step=STEP_PER_EPOCH, verbose=100)
        val_loss1, val_pred = validate(valid_loader, model, verbose=100)

        # save and print log
        endtime = time.time() - starttime
        print_log = "Epoch: {}/{} ".format(epoch + 1, NUM_EPOCH)
        print_log += "BCE 1: {:.3f} ".format(loss1)
        print_log += "val BCE 1: {:.3f} ".format(val_loss1)
        print_log += "Sec: {:.1f} \n".format(time.time()-starttime)
        train_log_epoch = [epoch+1, loss1, val_loss1, endtime]
        for i, col in enumerate(col_targets):
            score = metrics.log_loss(y_valid[:,i], val_pred[:,i], labels=[0,1])
            train_log_epoch.append(score)
            print_log += "val bce {}: {:.3f} ".format(col, score)
            
            score = metrics.roc_auc_score(y_valid[:,i], val_pred[:,i])
            train_log_epoch.append(score)
            print_log += "val auc {}: {:.3f} \n".format(col, score)
        train_log_epoch = pd.DataFrame([train_log_epoch], columns=log_columns)
        train_log = pd.concat([train_log, train_log_epoch])
        train_log.to_csv("{}/train_log_{}_fold{}.csv".format(output_dir, MODEL_NAME, fold + 1), index=False)
        print(print_log)
       
        # save weights
        if (epoch+1)%NUM_CYCLE==0:
            torch.save(model.state_dict(), "{}/weight_{}_epoch_{}_fold{}.pth".format(
                    output_dir, MODEL_NAME, epoch+1, fold + 1))
            torch.save(optimizer.state_dict(), "{}/optimizer_{}_epoch_{}_fold{}.pth".format(
                    output_dir, MODEL_NAME, epoch+1, fold + 1))

fold: 1
Step: 100/512 BCE 1: 0.265 Sec: 152.0 
Step: 200/512 BCE 1: 0.255 Sec: 300.0 
Step: 300/512 BCE 1: 0.248 Sec: 446.8 
Step: 400/512 BCE 1: 0.245 Sec: 597.6 
Step: 500/512 BCE 1: 0.241 Sec: 746.6 
Step: 512/512 BCE 1: 0.240 Sec: 764.2 
Step: 100/449 BCE 1: 0.238 Sec: 133.8 
Step: 200/449 BCE 1: 0.220 Sec: 262.4 
Step: 300/449 BCE 1: 0.223 Sec: 394.0 
Step: 400/449 BCE 1: 0.228 Sec: 526.3 
Step: 449/449 BCE 1: 0.225 Sec: 588.4 
Epoch: 1/1 BCE 1: 0.240 val BCE 1: 0.225 Sec: 1361.6 
val bce negative_exam_for_pe: 0.615 val auc negative_exam_for_pe: 0.635 
val bce indeterminate: 0.096 val auc indeterminate: 0.628 
val bce chronic_pe: 0.163 val auc chronic_pe: 0.586 
val bce acute_and_chronic_pe: 0.080 val auc acute_and_chronic_pe: 0.648 
val bce central_pe: 0.191 val auc central_pe: 0.736 
val bce leftsided_pe: 0.520 val auc leftsided_pe: 0.636 
val bce rightsided_pe: 0.549 val auc rightsided_pe: 0.656 
val bce rv_lv_ratio_gte_1: 0.368 val auc rv_lv_ratio_gte_1: 0.661 
val bce rv_lv_r

# Feature Extraction

In [20]:
def start_timer():
    global start_time
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()
    start_time = time.time()

In [21]:
def extract_feature(loader, model, verbose=10, fold=1):
    # make save dir
    save_dir = "{}/features_{}_fold{}".format(output_dir, MODEL_NAME, fold)
    print(save_dir)
    os.makedirs(save_dir, exist_ok=True)
    
    loss1_avr = AverageMeter()

    # switch to eval mode
    model .eval()

    starttime = time.time()
    # feature extraction
    preds = np.zeros([0, NUM_CLASS], np.float16)
    features = np.zeros([0, model.base_model.num_features], np.float16)
    exam_index = 0
    for i, (input1, _, _) in enumerate(loader):
        if DEBUG and exam_index>=20: break
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                input1 = input1.to(device, non_blocking=True)
                output, feature = model.feature(input1)

        preds = np.concatenate([preds, output.data.cpu().numpy()])
        features = np.concatenate([features, feature.data.cpu().numpy()])
        while df_train_exam['num_series'][exam_index]<=preds.shape[0]: # concat features by exam level and save
            num_series = df_train_exam['num_series'][exam_index]
            exam = df_train_exam[col_groupby][exam_index]
            if (exam_index+1)%verbose==0:
                print("{}/{}: exam: {}, len: {}, sec: {:.1f}".format(
                    exam_index+1, len(df_train_exam), exam, num_series, time.time()-starttime))
            np.save("{}/{}_pred.npy".format(save_dir, exam), preds[:num_series])
            np.save("{}/{}_feature.npy".format(save_dir, exam), features[:num_series])
            preds = np.copy(preds[num_series:])
            features = np.copy(features[num_series:])
            exam_index += 1
            if exam_index>=len(df_train_exam): break
        gc.collect()

In [22]:
# feature extraction
start_timer()

# build model
if MODEL_NAME=='b0':
    base_model = 'tf_efficientnet_b0_ns'
elif MODEL_NAME=='b2':
    base_model = 'tf_efficientnet_b2_ns'
model  = CNN_2D(base_model=base_model, num_classes=NUM_CLASS).cuda()

for param in model.parameters():
    param.grad = None

dataset_valid = ImageDataset(df_train, df_train_exam, df_train[col_targets].values)

for fold in range(NUM_FOLD):
    if fold+1 not in FOLD_LIST: continue
    valid_loader = DataLoader(dataset_valid,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=multiprocessing.cpu_count(),
                              pin_memory=True,
                              )
    
    model.load_state_dict(torch.load("{}/weight_{}_epoch_{}_fold{}.pth".format(output_dir, MODEL_NAME, NUM_EPOCH, fold+1)))
    extract_feature(valid_loader, model, verbose=10, fold=fold+1)

/mnt/disks/data6/rsna2020/output/features_b0_new_fold1
10/7279: exam: 3c5fd9c92057, len: 96, sec: 11.3
20/7279: exam: d25e9842c1b3, len: 141, sec: 23.3
